In [3]:
!pip uninstall -y langchain langchain-core langchain-community langchain-text-splitters langchain-classic requests


Found existing installation: langchain 0.3.27
Uninstalling langchain-0.3.27:
  Successfully uninstalled langchain-0.3.27
Found existing installation: langchain-core 0.3.75
Uninstalling langchain-core-0.3.75:
  Successfully uninstalled langchain-core-0.3.75
Found existing installation: langchain-community 0.3.16
Uninstalling langchain-community-0.3.16:
  Successfully uninstalled langchain-community-0.3.16
Found existing installation: langchain-text-splitters 0.3.9
Uninstalling langchain-text-splitters-0.3.9:
  Successfully uninstalled langchain-text-splitters-0.3.9
Found existing installation: requests 2.32.4
Uninstalling requests-2.32.4:
  Successfully uninstalled requests-2.32.4


In [4]:
!pip install \
  langchain==0.3.27 \
  langchain-core==0.3.75 \
  langchain-community==0.3.16 \
  langchain-text-splitters==0.3.9 \
  sentence-transformers \
  faiss-cpu \
  transformers \
  torch \
  requests==2.32.4


  Using cached langchain-0.3.27-py3-none-any.whl.metadata (7.8 kB)
  Using cached langchain_core-0.3.75-py3-none-any.whl.metadata (5.7 kB)
  Using cached langchain_community-0.3.16-py3-none-any.whl.metadata (2.9 kB)
  Using cached langchain_text_splitters-0.3.9-py3-none-any.whl.metadata (1.9 kB)
  Using cached requests-2.32.4-py3-none-any.whl.metadata (4.9 kB)
Using cached langchain-0.3.27-py3-none-any.whl (1.0 MB)
Using cached langchain_core-0.3.75-py3-none-any.whl (443 kB)
Using cached langchain_community-0.3.16-py3-none-any.whl (2.5 MB)
Using cached langchain_text_splitters-0.3.9-py3-none-any.whl (33 kB)
Using cached requests-2.32.4-py3-none-any.whl (64 kB)


In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document

from transformers import pipeline

print("✅ Everything is finally working")


✅ Everything is finally working


1. You ask a question
2. System searches your document
3. Finds most relevant parts
4. Adds them to a prompt
5. Sends that to the model


In [6]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("ABOUT_ME.txt")
docs = loader.load()

print(docs[0].page_content[:200])


PERSONAL PROFILE DOCUMENT

[IDENTITY]
Name: Urvika Singh
Preferred Name: Urvika
Location: India
Current Role: AI / ML Learner

[BACKGROUND


ENCODING-EMBEDIINGMODEL(MINILM)<CONTEXT BASED EMBEDDINGS OF DOC

In [7]:
splitter=RecursiveCharacterTextSplitter(chunk_size=200,chunk_overlap=20)
chunks=splitter.split_documents(docs)

embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
db=FAISS.from_documents(chunks,embeddings)

/tmp/ipython-input-3205509740.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.war

DECODING-GPT2 MODEL <TEXT GENERATION BASED ON DOC(RAG)COMAPRISION

In [8]:
generator=pipeline("text-generation",
    model="Qwen/Qwen2.5-0.5B-Instruct")


Device set to use cpu


In [9]:
BOT_STYLE = """
You are a helpful and emotionally intelligent assistant.

RULES:
-ONLY ANSWER THE QUESTIONS DO NOT EXPLAIN YOUR RESONING
-ID YOU DONT KNOW ABOUT ANYTHING JUST SAY I DONT KNOW .
-ALWAYS USE FRIENDLY TONE OR AS YOUR USER SOUNDS .
-USE CONTEXT IF SOMETHING IS SIMILAR LIKE BEHAVIOUR, ACADEMC OTHERWISE YOU ARE A FRIENDLY CHATBOT TRYING TO COMMUNICATE EASILY WITH THE THE USER.
-DO NOT HALLUCINATE.
- NEVER repeat the context.
- NEVER invent user background.
- If the user is emotional, respond warmly.
- Answer in plain text only
"""


" What am I learning? "

        ↓
Embedding model encodes it

        ↓
FAISS compares with stored vectors

        ↓
Most similar chunks returned


In [14]:
def rag_chat(query):
    docs = db.similarity_search(query, k=2)
    context = "\n".join([d.page_content for d in docs]) if docs else ""

    prompt = f"""
{BOT_STYLE}

<CONTEXT>
{context}
</CONTEXT>

<USER_QUESTION>
{query}
</USER_QUESTION>

<ASSISTANT_ANSWER>
"""

    output = generator(
        prompt,
        max_new_tokens=150,
        do_sample=True,
        temperature=0.8,
        repetition_penalty=1.3,
        no_repeat_ngram_size=3,
        return_full_text=False     # ✅ ADD THIS


    )[0]["generated_text"]


    # Keep only first clean reply
    for stop_word in ["```", "<", "System:", "Context:", "The answer should"]:
        if stop_word in output:
            response = output.split(stop_word)[0]

    return response.strip()


In [15]:
print("Chatbot is ready. Type 'exit' to stop.\n")

while True:
    query = input("You: ")

    if query.lower() == "exit":
        print("Bot: Bye 👋")
        break

    response = rag_chat(query)
    print("\nBot:", response, "\n")



Chatbot is ready. Type 'exit' to stop.

You: hi 

Bot: Hi! How can I help you today? What would be your request or problem that needs assistance? 

You: can you help me with something

Bot: Sure! What do you need assistance with? 

You: i am feeling low , can you suggest anything to do that i like

Bot: Absolutely! There's so much great stuff out there. How about trying something new? Maybe taking up painting or pottery?
It could be fun for all ages!
If not now, maybe later when your mood starts lifting again?

Let me know if this helps!

Good luck with whatever project comes next! 

You: i dont like to paint

Bot: painting can be fun! you could try painting your favorite things or even create something unique for yourself. remember that it's okay if art isn't perfect right now; just let go of expectations! 

You: ok thanks

Bot: I'm glad to help! What would you like assistance with today? Let me know if there's anything specific or not sure about. 

You: suggest me any project

Bot: C